# Tuning and training a Convolutional Neural Network for Intrusion Detection
In this laboratory, you will implement and tune a CNN model for binary network traffic classification. The CNN model will return a value between 0 and 1, which is the probability of the input flow of being malicious. 
You will select a range of hyperparameters to tune the CNN using Random search and you will train it on a dataset of benign traffic and DDoS attack traffic.

| <img src="./cnn.png" width="90%">  |
|--|
| CNN model for binary classification|

You will use a dataset of benign and various DDoS attacks from the CIC-DDoS2019 dataset (https://www.unb.ca/cic/datasets/ddos-2019.html).
The network traffic has been previously pre-processed in a way that packets are grouped in bi-directional traffic flows using the 5-tuple (source IP, destination IP, source Port, destination Port, protocol). Each flow is represented as a **100x20 array**, where the rows are the packets of the flow in chronological order, while each column is a packet-level feature in the following order:

| Feature nr.         | Feature Name |
|---------------------|---------------------|
| 00 | timestamp (IAT) | 
| 01 | packet_length (bytes)| 
| 02 | IP_flags_df (0/1) |
| 03 | IP_flags_mf (0/1) |
| 04 | IP_flags_rb (0/1) | 
| 05 | IP_frag_off (0/1) |
| 06 | protocols (integer) |
| 07 | TCP_length (bytes) |
| 08 | TCP_flags_ack (0/1) |
| 09 | TCP_flags_cwr (0/1) |
| 10 | TCP_flags_ece (0/1) |
| 11 | TCP_flags_fin (0/1) |
| 12 | TCP_flags_push (0/1) |
| 13 | TCP_flags_res (0/1) |
| 14 | TCP_flags_reset (0/1) |
| 15 | TCP_flags_syn (0/1) |
| 16 | TCP_flags_urg (0/1) |
| 17 | TCP_window_size (bytes) |
| 18 | UDP_length (bytes) |
| 19 | ICMP_type (code) |

In [15]:
# Author: Roberto Doriguzzi-Corin
# Project: Course on Network Intrusion and Anomaly Detection with Machine Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalMaxPooling2D, Flatten, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.regularizers import l1,l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
import os
import random as rn
import numpy as np
import logging
import time
from util_functions import *

# disable GPUs for test reproducibility
tf.config.set_visible_devices([], 'GPU')

SEED=0

DATASET_FOLDER = "./DOS2019"

X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5',channels=True)
X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5',channels=True)
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5',channels=True)

In [16]:
def compileModel(model,optimizer='sgd', lr=0.001):
    if optimizer == 'sgd':
        optimizer = SGD(learning_rate=lr, momentum=0.0)
    else:
        optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy']) 

# Model definition
The following method defines the CNN model with configurable hyperparameters. Each hyperparameter has a default value that can be set during the tuning process. In the following cell, your task is to finalise the model by adding a **GlobalMaxPooling2D** layer, a **Flatten** layer and a final **Dense** layer with Sigmoid activation function for binary classification. Additional, you may want to add **Dropout** regularisation to your model. Take inspiration from the [CNNvsMLP](./CNNvsMLP.ipynb) and the [Hyperparameters Tuning](../09-HyperparameterTuning/hyperparameter-tuning-lab.ipynb) notebooks.

You may notice that the **Convolutional** layer (Conv2D) has fixed hyperparameters' values (number of filters, kernel size, etc.). Change the code in a way that Conv2D can use the values listed in the definition of the *create_model* method.

In [ ]:
# Function to create the CNN model
def create_model(optimizer='adam', filters = 100, kernel_size=(3,3), strides=(1,1), padding='same',learning_rate = 0.001,dropout_rate=0.1):
    cnn_model = Sequential(name  = "cnn")

    ### MODIFY THE FOLLOWING CODE
    cnn_model.add(Conv2D(filters=100, kernel_size=(3,3), input_shape=X_train.shape[1:], data_format='channels_last', activation='relu', padding='same', strides=(1,1)))
    
    ### ADD YOUR CODE HERE ###
    cnn_model.add(Dropout(dropout_rate))
    cnn_model.add(MaxPooling2D((2,2), padding='valid'))
    
    cnn_model.add(Flatten())
    
    cnn_model.add(Dense(1, activation='sigmoid'))

    compileModel(cnn_model, optimizer,learning_rate)
    return cnn_model


# Random search
The code in the following cell implements *random search* to tune the hyperparameters of the CNN model. 

In the cell below, add the relevant hyperparameters for the CNN to the **param_dist** dictionary. The tunable hyperparameters are those available in definition of the *create_model* above. Remember to use the *uniform* method for generating floating point values (e.g., for the **learning_rate**), use *randint* for generating the integer hyperparameters (e.g., the **number of filters**), while use lists for multi-dimensional hyperparameters (e.g., **kernel_size**). 

In [ ]:
from scipy.stats import uniform, randint
k=2 # number of folds for cross-validation
PATIENCE = 10

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)

# Define the hyperparameters to tune and their possible values
param_dist = {
    'learning_rate' : uniform(0.0001, 0.001),
    'dropout_rate': uniform(0.1, 0.3),
    'optimizer': ['adam', 'sgd'],
    'filters': randint(16, 64),
    'kernel_size': [(2,2),(2,3),(3,3)],
    'padding': ['same', 'valid'],
    'strides': [(1,1), (2,2)],

}

# Perform grid search with 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, cv=k, random_state=SEED)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE, restore_best_weights=True)
start_time = time.time()
random_search_result = random_search.fit(X_train, y_train,epochs=100, validation_data=(X_val, y_val),callbacks= [early_stopping])
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)

# Print the best parameters and corresponding accuracy
print("Best parameters found: ", random_search_result.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(random_search_result.best_score_))

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test accuracy of the best model: {:.2f}".format(test_accuracy))

/var/folders/td/g96p99dn30gb034k_t5kqbsr0000gn/T/ipykernel_26194/3895092144.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)
2024-11-13 17:22:59.268037: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100
19/19 [==============================] - 1s 51ms/step - loss: 103.1726 - accuracy: 0.7339 - val_loss: 0.5912 - val_accuracy: 0.6863
Epoch 2/100
19/19 [==============================] - 1s 40ms/step - loss: 0.4730 - accuracy: 0.8533 - val_loss: 0.7287 - val_accuracy: 0.8480
Epoch 3/100
19/19 [==============================] - 1s 38ms/step - loss: 0.7696 - accuracy: 0.8430 - val_loss: 0.4986 - val_accuracy: 0.8186
Epoch 4/100
19/19 [==============================] - 1s 39ms/step - loss: 0.4425 - accuracy: 0.8397 - val_loss: 0.4989 - val_accuracy: 0.8186
Epoch 5/100
19/19 [==============================] - 1s 38ms/step - loss: 0.4465 - accuracy: 0.8451 - val_loss: 0.4636 - val_accuracy: 0.7966
Epoch 6/100
19/19 [==============================] - 1s 38ms/step - loss: 0.4282 - accuracy: 0.8441 - val_loss: 0.5367 - val_accuracy: 0.7843
Epoch 7/100
19/19 [==============================] - 1s 40ms/step - loss: 0.4275 - accuracy: 0.8430 - val_loss: 0.5061 - val_accuracy: 0.8358
Epoc